In [7]:
import time
from datetime import datetime
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.functional import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

# Memuat model BERT dan tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Kamus kata untuk kategori Prioritas dan Non Prioritas
prioritas_kamus = {
    'urgensi': [
        'penting', 'segera', 'kritis', 'darurat', 'mendesak', 'harus segera',
        'segera ditangani', 'harus selesai', 'urgent', 'emergency',
        'prioritas', 'krusial', 'vital', 'genting', 'tak bisa ditunda'
    ],
    'bimbingan_proyek': [
        'bimbingan', 'proyek', 'pengembangan', 'tugas besar',
        'tugas akhir', 'skripsi', 'tesis', 'proposal', 'pekerjaan',
        'deadline', 'dokumen proyek', 'pengarahan', 'implementasi'
    ],
    'konsultasi': [
        'diskusi', 'konsultasi', 'masalah', 'solusi', 'pembahasan',
        'tanya jawab', 'saran', 'feedback', 'review', 'evaluasi',
        'penyelesaian', 'klarifikasi', 'mencari solusi', 'problem solving'
    ],
    'ajukan masalah': [
        'terdapat masalah', 'kendala', 'kesalahan', 'error', 'bug',
        'gangguan', 'tidak berfungsi', 'trouble', 'issue', 'kerusakan',
        'komplain', 'error code', 'problem', 'tidak berjalan',
        'malfungsi', 'terhambat', 'crash', 'perlu perbaikan'
    ]
}

urgency_kamus = {
    5: ['segera', 'kritis', 'darurat', 'sangat penting', 'terburu-buru', 'urgent', 'harus segera'],
    4: ['mendesak', 'cepat', 'waktu terbatas', 'harus selesai', 'perlu segera', 'segera ditangani', 'prioritas tinggi'],
    3: ['normal', 'standar', 'cukup', 'biasa', 'wajar', 'tidak terlalu mendesak', 'terjadwal', 'routine'],
    2: ['santai', 'tidak mendesak', 'bisa ditunda', 'fleksibel', 'tidak segera', 'bisa tunggu'],
    1: ['kapan-kapan', 'relaksasi', 'tidak penting', 'sangat santai', 'tidak perlu segera', 'bisa ditunda']
}

# Kelas untuk pengajuan
class Pengajuan:
    def __init__(self, teks, waktu_pengajuan, urgensi, kecocokan, kategori, kategori_prioritas):
        self.teks = teks
        self.waktu_pengajuan = waktu_pengajuan
        self.urgensi = urgensi
        self.kecocokan = kecocokan
        self.kategori = kategori
        self.kategori_prioritas = kategori_prioritas

# Fungsi untuk menghitung kesamaan cosine
def get_similarity(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs2 = tokenizer(text2, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    embeddings1 = outputs1.pooler_output
    embeddings2 = outputs2.pooler_output

    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity.item()

# Fungsi untuk menghitung kesamaan berdasarkan kamus kata
def hitung_kesamaan(teks, kamus):
    teks = teks.lower()
    score = sum(1 for kategori, kata_kunci in kamus.items() for kata in kata_kunci if kata in teks)
    return score

# Fungsi untuk menentukan tingkat urgensi berdasarkan teks
def tentukan_urgensi(teks):
    teks = teks.lower()
    max_score = 0
    urgency_level = 3
    for level, kata_kunci in urgency_kamus.items():
        score = sum(1 for kata in kata_kunci if kata in teks)
        if score > max_score:
            max_score = score
            urgency_level = level

    # Penalti jika teks terlalu pendek atau tidak bermakna
    if len(teks.split()) < 3 or max_score == 0:
        urgency_level = 1  # Non mendesak
    return urgency_level

# Daftar pengajuan
daftar_pengajuan = []

# Fungsi untuk menambahkan pengajuan
def tambah_pengajuan(teks, kategori_teks, tipe_pertemuan):
    if len(teks.split()) < 3:
        print("Deskripsi terlalu pendek atau tidak jelas. Mohon masukkan deskripsi yang lebih spesifik.")
        return

    # Cek jika pengajuan dengan teks yang sama sudah ada
    for pengajuan in daftar_pengajuan:
        if pengajuan.teks == teks:
            print("Pengajuan dengan deskripsi yang sama sudah ada.")
            return

    urgency = tentukan_urgensi(teks)
    similarity_score = get_similarity(teks, kategori_teks)
    timestamp = datetime.now()
    match_count = hitung_kesamaan(teks, prioritas_kamus)

    # Tentukan kategori prioritas
    if match_count >= 4 or urgency >= 4:
        kategori_prioritas = "Prioritas"
    else:
        kategori_prioritas = "Non Prioritas"

    pengajuan = Pengajuan(teks, timestamp, urgency, match_count, tipe_pertemuan, kategori_prioritas)
    daftar_pengajuan.append(pengajuan)

    # Mengurutkan pengajuan berdasarkan urgensi, timestamp, dan jumlah kata yang cocok
    daftar_pengajuan.sort(key=lambda x: (-x.urgensi, x.waktu_pengajuan, -x.kecocokan))

# Fungsi untuk menampilkan daftar pengajuan
def tampilkan_pengajuan():
    if not daftar_pengajuan:
        result_label.value = "Belum ada pengajuan."
        return

    result_text = ""
    for i, pengajuan in enumerate(daftar_pengajuan, start=1):
        result_text += (
            f"{i}. [{pengajuan.kategori}] Urgensi: {pengajuan.urgensi}, "
            f"Kategori: {pengajuan.kategori_prioritas}, "
            f"Waktu: {pengajuan.waktu_pengajuan}, Cocok: {pengajuan.kecocokan} kata, "
            f"Deskripsi: {pengajuan.teks}\n"
        )
    result_label.value = result_text

# Fungsi untuk mengolah input dari antarmuka
def on_button_click(b):
    text_input = text_input1.value
    meeting_type = meeting_type_dropdown.value

    if meeting_type == 'Bimbingan Proyek':
        kategori_teks = "Tugas pengembangan proyek yang mendesak"
    elif meeting_type == 'Konsultasi':
        kategori_teks = "Diskusi tentang masalah teknis"
    elif meeting_type == 'Ajukan Masalah':
        kategori_teks = "Masalah yang perlu segera diselesaikan"
    elif meeting_type == 'Penyetoran Tugas':
        kategori_teks = "Tugas rutin atau penyetoran pekerjaan"

    tambah_pengajuan(text_input, kategori_teks, meeting_type)
    tampilkan_pengajuan()

# Input widgets
text_input1 = widgets.Textarea(
    description='Deskripsi:',
    placeholder='Masukkan deskripsi...',
    layout=widgets.Layout(width='90%', height='100px')
)

meeting_type_dropdown = widgets.Dropdown(
    options=['Bimbingan Proyek', 'Konsultasi', 'Ajukan Masalah', 'Penyetoran Tugas'],
    description='Tipe Pertemuan:'
)

button = widgets.Button(description="Tambahkan Pengajuan")
button.on_click(on_button_click)

result_label = widgets.Textarea(
    value="Belum ada pengajuan.",
    layout=widgets.Layout(width='90%', height='200px')
)

# Menampilkan antarmuka pengguna
display(text_input1, meeting_type_dropdown, button, result_label)


Textarea(value='', description='Deskripsi:', layout=Layout(height='100px', width='90%'), placeholder='Masukkan…

Dropdown(description='Tipe Pertemuan:', options=('Bimbingan Proyek', 'Konsultasi', 'Ajukan Masalah', 'Penyetor…

Button(description='Tambahkan Pengajuan', style=ButtonStyle())

Textarea(value='Belum ada pengajuan.', layout=Layout(height='200px', width='90%'))